# The world's most remote country capital
### Your challenge is to find the world's most remote country capital.

Find a list of countries and their capitals and the longitude and latitude of the capital. Decide on a way to measure the remoteness of a capital. For example, you could use the distance to its nearest country capital. Or, if that's too rough, something a bit more sophisticated.

Keep in mind that the capitals lie on a curved surface, so to calculate distances you will have to use something like the **Haversine formula**.

You might need to make some decisions. For example, it might not be clear whether certain territories count as countries, or what should be counted as the capital of some countries. Clearly explain any decisions you make.

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:,.2f}'.format

# Acquired from https://www.kaggle.com/viswanathanc/world-cities-datasets
df = pd.read_csv('worldcities.csv')
#print(df.head())

In [2]:
from math import pi, sin, cos, atan2, sqrt

def haversine(lat1, lon1, lat2, lon2):
    '''Taken from https://www.movable-type.co.uk/scripts/latlong.html'''
    R = 6371*10**3 # metres
    φ1 = lat1 * pi/180 # φ, λ in radians
    φ2 = lat2 * pi/180
    Δφ = (lat2-lat1) * pi/180
    Δλ = (lon2-lon1) * pi/180
    
    a = sin(Δφ/2) * sin(Δφ/2) + cos(φ1) * cos(φ2) * sin(Δλ/2) * sin(Δλ/2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c; # in metres
    return d/1000 # in kms

In [3]:
capitals = df[df['capital'] == 'primary']
capitals = capitals[['city','lat','lng']]

In [13]:
# Iterate over rows, comparing each row to the rest, looking for the minimum distance
capitals['closest'] = 99999999
for idx, row_x in capitals.iterrows():
    capitals['closest'] = capitals.apply(lambda row_y: 
                                 min(row_y['closest'], haversine(row_x['lat'],row_x['lng'],row_y['lat'],row_y['lng'])
                                 if row_x['city'] != row_y['city']
                                 else row_y['closest']
                                ), axis=1)

capitals = capitals.sort_values(by='closest', ascending=False)
capitals.reset_index(drop=True, inplace=True)
print(capitals.head())
capitals.to_csv('capitals_output.csv')

         city    lat    lng  closest
0    Canberra -35.28 149.13 2,326.96
1  Wellington -41.30 174.78 2,326.96
2        Dili  -8.56 125.58 1,904.68
3   Reykjavík  64.15 -21.95 1,497.12
4    Brasília -15.78 -47.92 1,463.15


PermissionError: [Errno 13] Permission denied: 'capitals_output.csv'